2022년 7월까지의 데이터를 학습하고 8,9,10월의 데이터를 예측할것

In [2]:
import pandas as pd
import numpy as np
from haversine import haversine
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow import keras
import numpy as np
from numpy import concatenate
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
import random
import itertools
from sklearn import linear_model

함수정의

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
      cols.append(df.shift(i))
      names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
      cols.append(df.shift(-i))
      if i == 0:
          names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
      else:
          names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
      agg.dropna(inplace=True)
  return agg

In [4]:
#기준 아파트로부터 'near_range' km 내에 존재하는 아파트 코드 반환
def finding_near_apt(gizun,near_range):
    start = (float(gizun['GPS_Y']),float(gizun['GPS_X']))
    near_apts = []
    for j in df_range.index:
        goal = (float(df_range['GPS_Y'][j]),float(df_range['GPS_X'][j]))
        if haversine(start, goal) < near_range:
            near_apts.append(df_range['apart_code'][j])
    near_df = pd.DataFrame()
    near_df['apart_code'] = near_apts
    return near_df

In [5]:
#사용하지 않을 과거 데이터 제거(윈도우 사이즈가 3일경우)
def delete_unuse_var(reframed):
    var_count = int(len(reframed.columns)/4)
    reframed.drop(reframed.columns[var_count*2:(var_count*3) - 1], axis=1, inplace=True)
    reframed.drop(reframed.columns[var_count:(var_count*2) - 1], axis=1, inplace=True)
    reframed.drop(reframed.columns[0:var_count - 1], axis=1, inplace=True)
    return reframed

In [6]:
#데이터프레임에서 일정한 컬럼만 추출하고 값 형태로 반환
def get_float_values(df):
    df = df[['YYYYMM','sale_type', 'area_m3', 'floor', 'const_year', 
    'parking','crime_rate', 'building_year', 'market_dist','busstop_dist', 'subway_dist', 
    'kinder_dist', 'hospital_dist', 'ASPI','YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 
    'GNI', 'CPI','avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold','near_element', 
    'near_middle', 'near_high', 'generation','const_rank','prices']]
    values = df.values
    values = values.astype('float32')
    return values

In [7]:
#코드들로부터 값을 추출하고 합치기
def get_from_codes(codes):
    for j in codes:
        df1 = globals()['df_'+j]
        values = get_float_values(df1)
        reframed = series_to_supervised(values, 3, 1)
        reframed = delete_unuse_var(reframed)
        global df_all
        df_all = pd.concat([reframed,df_all])
    return df_all

In [8]:
#추가하고 싶은 달을 입력
def make_future_data(new_month):
    new_df = df.drop_duplicates(subset = ['apart_code','area_m3'])
    new_df['YYYYMM'] = 202200 + new_month
    new_df['crime_rate'] = np.nan
    new_df['ASPI'] = np.nan
    new_df['YYYY'] = 2022
    new_df['MM'] = new_month
    new_df['HAI'] = np.nan
    new_df['PIR'] = np.nan
    new_df['LIR'] = np.nan
    new_df['GDP'] = np.nan
    new_df['GDI'] = np.nan
    new_df['GNI'] = np.nan
    new_df['CPI'] = np.nan
    new_df['avg_interest'] = np.nan
    new_df['kospi'] = np.nan
    new_df['M2'] = np.nan
    new_df['house_CCSI'] = np.nan
    new_df['unsold'] = np.nan
    #가격은 뭘로 할건지
    new_df['prices'] = 0 
    return new_df

In [9]:
def fill_nan(X_list,Y,df_a):
    lin_reg = linear_model.LinearRegression()
    x = df_a.dropna(axis=0)[X_list]
    y = df_a.dropna(axis=0)[Y]
    lin_reg_model = lin_reg.fit(x,y)
    y_pred = lin_reg_model.predict(df_a.loc[:, X_list])
    df_a[Y] = np.where(df_a[Y].isnull(),pd.Series(y_pred.flatten()),df_a[Y])
    return df_a

파일 읽기

In [10]:
df = pd.read_csv('final6.csv',encoding='cp949')

In [11]:
#결측값 제거
df = df[df['sale_type'].notna()]
df = df[df['parking'].notna()]
df = df[df['HAI'].notna()]
df = df[df['M2'].notna()]
#정성적변수 정량적 변수로 변환
df.loc[df['sale_type'] == '분양', 'sale_type'] = 0
df.loc[df['sale_type'] == '임대+분양', 'sale_type'] = 1
df.loc[df['sale_type'] == '임대', 'sale_type'] = 2
df.loc[df['sale_type'] == '영구임대', 'sale_type'] = 3
df['area_m3']=df['area_m3'].round(-1)
df = df.astype({'prices':'float64'})
df['prices']=df.groupby(['YYYYMM','apart_code','area_m3'])['prices'].transform('mean')
df['apart_code_area'] = df['apart_code'].map(str) +'_' +df['area_m3'].map(int).map(str)
df_a = df
exiting_code = df['apart_code'].unique()
df_vary = df.drop_duplicates(['apart_code'])
df_range = df_vary[['apart_code','GPS_X','GPS_Y']]
many_df = pd.read_csv('서울시아파트(기준).csv',encoding='cp949')
many_df = many_df[many_df['k-아파트코드'].isin(exiting_code)]

예측하고 싶은 값 추가

In [12]:
#df_predction_8 = df.drop_duplicates(subset = ['apart_code','area_m3'])

In [13]:
'''
df_predction_8['YYYYMM'] = 202208
df_predction_8['crime_rate'] = np.nan
df_predction_8['ASPI'] = np.nan
df_predction_8['YYYY'] = 2022
df_predction_8['MM'] = 8
df_predction_8['HAI'] = np.nan
df_predction_8['PIR'] = np.nan
df_predction_8['LIR'] = np.nan
df_predction_8['GDP'] = np.nan
df_predction_8['GDI'] = np.nan
df_predction_8['GNI'] = np.nan
df_predction_8['CPI'] = np.nan
df_predction_8['avg_interest'] = np.nan
df_predction_8['kospi'] = np.nan
df_predction_8['M2'] = np.nan
df_predction_8['house_CCSI'] = np.nan
df_predction_8['unsold'] = np.nan
#가격은 뭘로 할건지
df_predction_8['prices'] = 0 
'''

"\ndf_predction_8['YYYYMM'] = 202208\ndf_predction_8['crime_rate'] = np.nan\ndf_predction_8['ASPI'] = np.nan\ndf_predction_8['YYYY'] = 2022\ndf_predction_8['MM'] = 8\ndf_predction_8['HAI'] = np.nan\ndf_predction_8['PIR'] = np.nan\ndf_predction_8['LIR'] = np.nan\ndf_predction_8['GDP'] = np.nan\ndf_predction_8['GDI'] = np.nan\ndf_predction_8['GNI'] = np.nan\ndf_predction_8['CPI'] = np.nan\ndf_predction_8['avg_interest'] = np.nan\ndf_predction_8['kospi'] = np.nan\ndf_predction_8['M2'] = np.nan\ndf_predction_8['house_CCSI'] = np.nan\ndf_predction_8['unsold'] = np.nan\n#가격은 뭘로 할건지\ndf_predction_8['prices'] = 0 \n"

In [14]:
#df_prediction_8 과 df_all을 결합
df_predction_8 = make_future_data(8)
df_a = pd.concat([df_a,df_predction_8], ignore_index=False)
df_a = df_b
df_a = fill_nan(['YYYYMM', 'YYYY', 'MM','GPS_X','GPS_Y'],'crime_rate',df_a)
filling_list = ['ASPI','HAI', 'PIR', 'LIR', 'GDP', 'GDI', 'GNI', 'CPI','avg_interest', 'kospi', 'M2','house_CCSI']
for i in filling_list:
    df_a = fill_nan(['YYYYMM', 'YYYY', 'MM'],i,df_a)
df_a = fill_nan(['YYYYMM', 'YYYY', 'MM'],'unsold',df_a)

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

NameError: name 'df_b' is not defined

In [ ]:
df_a

,apart_code,YYYYMM,apart__name,sigungu,town_name,address,sale_type,area_m3,floor,const_year,...,house_CCSI,unsold,prices,near_element,near_middle,near_high,generation,const,const_rank,apart_code_area
0,A10024240,201201,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,100.0,1,1978,...,106.000000,1890.000000,79666.666667,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_100
1,A10024240,201201,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,100.0,11,1978,...,106.000000,1890.000000,79666.666667,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_100
2,A10024240,201201,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,80.0,6,1978,...,106.000000,1890.000000,64000.000000,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_80
3,A10024240,201201,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,100.0,3,1978,...,106.000000,1890.000000,79666.666667,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_100
4,A10024240,201202,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,100.0,7,1978,...,111.300000,1703.000000,80170.000000,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725040,A10023991,202208,태릉해링턴플레이스,서울특별시 노원구 공릉동,공릉동,서울특별시 노원구 공릉로34길 86,1,60.0,12,2022,...,121.565028,-1315.283129,0.000000,0.478162,0.465578,0.792782,1308.0,효성중공업,4,A10023991_60
725041,A10023991,202208,태릉해링턴플레이스,서울특별시 노원구 공릉동,공릉동,서울특별시 노원구 공릉로34길 86,1,70.0,19,2022,...,121.565028,-1315.283129,0.000000,0.478162,0.465578,0.792782,1308.0,효성중공업,4,A10023991_70
725044,A10024359,202208,삼정아트테라스정동,서울특별시 중구 정동,정동,서울특별시 중구 서소문로 89-20,0,30.0,2,2020,...,121.565028,-1315.283129,0.000000,0.177679,0.349024,0.260624,168.0,삼정기업,4,A10024359_30
725045,A10024359,202208,삼정아트테라스정동,서울특별시 중구 정동,정동,서울특별시 중구 서소문로 89-20,0,10.0,8,2020,...,121.565028,-1315.283129,0.000000,0.177679,0.349024,0.260624,168.0,삼정기업,4,A10024359_10


실제로 돌아가는 모델

8월 추가

In [ ]:
#6분정도 걸리는듯
#exiting_code = df['apart_code'].unique()
test_codes = random.sample(list(exiting_code),10)
for i in test_codes:
    gizun_code = i
    gizun = df_range[df_range['apart_code']==gizun_code]
    #변수 변경시 거리 조정
    near_df = finding_near_apt(gizun,2)
    near_df = pd.merge(near_df,many_df,left_on='apart_code',right_on='k-아파트코드',how='left')
    near_df = near_df[['apart_code','k-전체세대수']]
    gizun_count = near_df[near_df['apart_code']==gizun_code].iloc[0][1]
    indexNames = near_df[near_df['k-전체세대수'] < gizun_count].index
    near_df.drop(indexNames , inplace=True)
    df = df_a
    df = pd.merge(near_df,df,on='apart_code')
    df.pop('k-전체세대수')
    df = df.drop_duplicates(subset=['apart_code_area','YYYYMM'])
    df = df.sort_values(by=["apart_code_area", "YYYYMM"], ascending=[True, True])
    df = df[['apart_code_area', 'YYYYMM','sale_type', 'area_m3', 'floor', 'const_year', 
    'parking','crime_rate', 'building_year', 'market_dist','busstop_dist', 'subway_dist', 
    'kinder_dist', 'hospital_dist', 'ASPI','YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 
    'GNI', 'CPI','avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold','near_element', 
    'near_middle', 'near_high', 'generation','const_rank','prices']]
    codes = df['apart_code_area'].unique()
    groups = df.groupby(df.apart_code_area)
    for j in codes:
        globals()['df_'+j] = groups.get_group(j)
    gizun_code_areas = df[df['apart_code_area'].str.contains(gizun_code)]['apart_code_area'].unique()
    for k in gizun_code_areas:
        gizun_code_area = k
        gizun_df = globals()['df_'+gizun_code_area]
        df_all = pd.DataFrame()
        deleted_code = 'df_'+gizun_code_area
        codes = np.delete(codes, np.where(codes == deleted_code))
        df_all = get_from_codes(codes)
        train_set = df_all[df_all['var1(t)']<202208]
        df_test = gizun_df
        values = get_float_values(df_test)
        reframed = series_to_supervised(values, 3, 1)
        reframed = delete_unuse_var(reframed)
        #날짜를 사용해 df와 df_prediction_8 구분
        test_set = reframed[reframed['var1(t)']>=202208]
        df_set = pd.concat([train_set,test_set])
        train_days =  len(train_set)
        values= df_set.values
        scaled = scaler.fit_transform(values)
        values = scaled
        train_days =  len(train_set)
        #테스트 데이터 얼마나 사용할 건지
        train = values[:train_days]
        test = values[train_days:]
        # split into input and outputs
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        history = model.fit(train_X, train_y, epochs=50, batch_size=72, 
                validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # make a prediction
        try:
            yhat = model.predict(test_X)
            test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
            inv_yhat = concatenate((test_X[:, 0:],yhat), axis=1)
            # invert scaling for actual
            inv_yhat = scaler.inverse_transform(inv_yhat)
            inv_yhat = inv_yhat[:,-1]
            #만약 해당 값을 데이터 프레임에 넣고 싶다면
            df_a['prices'][(df_a['apart_code_area']==k)&(df_a['YYYYMM']==202208)] = float(inv_yhat[0])
        except:
            #만약 해당 아파트의 2020년 이후 거래내역이 존재하지 않을 경우 오류 발생
            print('오류 발생') 

Epoch 1/50
72/72 - 3s - loss: 0.1024 - val_loss: 0.7729 - 3s/epoch - 38ms/step
Epoch 2/50
72/72 - 0s - loss: 0.0614 - val_loss: 0.8121 - 289ms/epoch - 4ms/step
Epoch 3/50
72/72 - 0s - loss: 0.0353 - val_loss: 0.7944 - 249ms/epoch - 3ms/step
Epoch 4/50
72/72 - 0s - loss: 0.0261 - val_loss: 0.7600 - 258ms/epoch - 4ms/step
Epoch 5/50
72/72 - 0s - loss: 0.0255 - val_loss: 0.7539 - 274ms/epoch - 4ms/step
Epoch 6/50
72/72 - 0s - loss: 0.0272 - val_loss: 0.6513 - 292ms/epoch - 4ms/step
Epoch 7/50
72/72 - 0s - loss: 0.0321 - val_loss: 0.6120 - 289ms/epoch - 4ms/step
Epoch 8/50
72/72 - 0s - loss: 0.0342 - val_loss: 0.7013 - 263ms/epoch - 4ms/step
Epoch 9/50
72/72 - 0s - loss: 0.0332 - val_loss: 0.7467 - 281ms/epoch - 4ms/step
Epoch 10/50
72/72 - 0s - loss: 0.0279 - val_loss: 0.6947 - 296ms/epoch - 4ms/step
Epoch 11/50
72/72 - 0s - loss: 0.0253 - val_loss: 0.6108 - 304ms/epoch - 4ms/step
Epoch 12/50
72/72 - 0s - loss: 0.0278 - val_loss: 0.6284 - 298ms/epoch - 4ms/step
Epoch 13/50
72/72 - 0s - lo

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/50
72/72 - 3s - loss: 0.0944 - val_loss: 0.6445 - 3s/epoch - 36ms/step
Epoch 2/50
72/72 - 0s - loss: 0.0556 - val_loss: 0.8189 - 314ms/epoch - 4ms/step
Epoch 3/50
72/72 - 0s - loss: 0.0307 - val_loss: 0.8782 - 334ms/epoch - 5ms/step
Epoch 4/50
72/72 - 0s - loss: 0.0254 - val_loss: 0.8747 - 320ms/epoch - 4ms/step
Epoch 5/50
72/72 - 0s - loss: 0.0249 - val_loss: 0.8722 - 327ms/epoch - 5ms/step
Epoch 6/50
72/72 - 0s - loss: 0.0275 - val_loss: 0.7282 - 288ms/epoch - 4ms/step
Epoch 7/50
72/72 - 0s - loss: 0.0303 - val_loss: 0.7515 - 267ms/epoch - 4ms/step
Epoch 8/50
72/72 - 0s - loss: 0.0318 - val_loss: 0.7979 - 283ms/epoch - 4ms/step
Epoch 9/50
72/72 - 0s - loss: 0.0396 - val_loss: 0.6981 - 260ms/epoch - 4ms/step
Epoch 10/50
72/72 - 0s - loss: 0.0345 - val_loss: 0.7674 - 269ms/epoch - 4ms/step
Epoch 11/50
72/72 - 0s - loss: 0.0288 - val_loss: 0.8139 - 273ms/epoch - 4ms/step
Epoch 12/50
72/72 - 0s - loss: 0.0231 - val_loss: 0.8829 - 295ms/epoch - 4ms/step
Epoch 13/50
72/72 - 0s - lo

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/50
48/48 - 3s - loss: 0.1394 - val_loss: 0.5313 - 3s/epoch - 66ms/step
Epoch 2/50
48/48 - 0s - loss: 0.0793 - val_loss: 0.6120 - 292ms/epoch - 6ms/step
Epoch 3/50
48/48 - 0s - loss: 0.0621 - val_loss: 0.5820 - 268ms/epoch - 6ms/step
Epoch 4/50
48/48 - 0s - loss: 0.0480 - val_loss: 0.5498 - 266ms/epoch - 6ms/step
Epoch 5/50
48/48 - 0s - loss: 0.0354 - val_loss: 0.5043 - 298ms/epoch - 6ms/step
Epoch 6/50
48/48 - 0s - loss: 0.0269 - val_loss: 0.4660 - 273ms/epoch - 6ms/step
Epoch 7/50
48/48 - 0s - loss: 0.0255 - val_loss: 0.4415 - 200ms/epoch - 4ms/step
Epoch 8/50
48/48 - 0s - loss: 0.0227 - val_loss: 0.4174 - 222ms/epoch - 5ms/step
Epoch 9/50
48/48 - 0s - loss: 0.0264 - val_loss: 0.3927 - 271ms/epoch - 6ms/step
Epoch 10/50
48/48 - 0s - loss: 0.0253 - val_loss: 0.3826 - 255ms/epoch - 5ms/step
Epoch 11/50
48/48 - 0s - loss: 0.0226 - val_loss: 0.3728 - 334ms/epoch - 7ms/step
Epoch 12/50
48/48 - 0s - loss: 0.0268 - val_loss: 0.3927 - 245ms/epoch - 5ms/step
Epoch 13/50
48/48 - 0s - lo

...

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/50
20/20 - 3s - loss: 0.0697 - val_loss: 0.1412 - 3s/epoch - 127ms/step
Epoch 2/50
20/20 - 0s - loss: 0.0801 - val_loss: 0.2697 - 111ms/epoch - 6ms/step
Epoch 3/50
20/20 - 0s - loss: 0.0406 - val_loss: 0.3253 - 110ms/epoch - 5ms/step
Epoch 4/50
20/20 - 0s - loss: 0.0723 - val_loss: 0.2059 - 110ms/epoch - 6ms/step
Epoch 5/50
20/20 - 0s - loss: 0.0494 - val_loss: 0.1618 - 120ms/epoch - 6ms/step
Epoch 6/50
20/20 - 0s - loss: 0.0768 - val_loss: 0.2604 - 114ms/epoch - 6ms/step
Epoch 7/50
20/20 - 0s - loss: 0.0275 - val_loss: 0.2875 - 90ms/epoch - 4ms/step
Epoch 8/50
20/20 - 0s - loss: 0.0462 - val_loss: 0.2197 - 92ms/epoch - 5ms/step
Epoch 9/50
20/20 - 0s - loss: 0.0479 - val_loss: 0.1583 - 92ms/epoch - 5ms/step
Epoch 10/50
20/20 - 0s - loss: 0.0660 - val_loss: 0.2402 - 99ms/epoch - 5ms/step
Epoch 11/50
20/20 - 0s - loss: 0.0224 - val_loss: 0.2556 - 111ms/epoch - 6ms/step
Epoch 12/50
20/20 - 0s - loss: 0.0281 - val_loss: 0.2245 - 88ms/epoch - 4ms/step
Epoch 13/50
20/20 - 0s - loss: 

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/50
20/20 - 3s - loss: 0.0910 - val_loss: 0.1711 - 3s/epoch - 128ms/step
Epoch 2/50
20/20 - 0s - loss: 0.0503 - val_loss: 0.2035 - 103ms/epoch - 5ms/step
Epoch 3/50
20/20 - 0s - loss: 0.0573 - val_loss: 0.1743 - 94ms/epoch - 5ms/step
Epoch 4/50
20/20 - 0s - loss: 0.0637 - val_loss: 0.2539 - 98ms/epoch - 5ms/step
Epoch 5/50
20/20 - 0s - loss: 0.0293 - val_loss: 0.2763 - 96ms/epoch - 5ms/step
Epoch 6/50
20/20 - 0s - loss: 0.0414 - val_loss: 0.2163 - 96ms/epoch - 5ms/step
Epoch 7/50
20/20 - 0s - loss: 0.0508 - val_loss: 0.2112 - 106ms/epoch - 5ms/step
Epoch 8/50
20/20 - 0s - loss: 0.0510 - val_loss: 0.2589 - 108ms/epoch - 5ms/step
Epoch 9/50
20/20 - 0s - loss: 0.0279 - val_loss: 0.2874 - 103ms/epoch - 5ms/step
Epoch 10/50
20/20 - 0s - loss: 0.0425 - val_loss: 0.2226 - 98ms/epoch - 5ms/step
Epoch 11/50
20/20 - 0s - loss: 0.0406 - val_loss: 0.2164 - 100ms/epoch - 5ms/step
Epoch 12/50
20/20 - 0s - loss: 0.0433 - val_loss: 0.2649 - 102ms/epoch - 5ms/step
Epoch 13/50
20/20 - 0s - loss: 

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
reframed

NameError: name 'reframed' is not defined

In [ ]:
df_a[(df_a['apart_code_area']==k)&(df_a['YYYYMM']==202208)]

,apart_code,YYYYMM,apart__name,sigungu,town_name,address,sale_type,area_m3,floor,const_year,...,house_CCSI,unsold,prices,near_element,near_middle,near_high,generation,const,const_rank,apart_code_area
525401,A13785302,202208,방배우성,서울특별시 서초구 방배동,방배동,서울특별시 서초구 방배선행길 1,0,80.0,5,1991,...,121.565028,-1315.283129,0.0,0.468301,0.913378,0.766222,468.0,우성건설,4,A13785302_80


In [ ]:
test_codes

['A13204409',
 'A13522006',
 'A14320002',
 'A13778209',
 'A11087601',
 'A15609502',
 'A13583301',
 'A15080205',
 'A10045002',
 'A13785302']

In [ ]:
for i in test_codes:
    print(df_a[['prices','apart_code','apart_code_area']][(df_a['apart_code']==i)&(df_a['YYYYMM']==202208)])

       prices apart_code apart_code_area
95341     0.0  A13204409   A13204409_130
95342     0.0  A13204409    A13204409_80
95343     0.0  A13204409    A13204409_60
        prices apart_code apart_code_area
151617     0.0  A13522006    A13522006_40
151618     0.0  A13522006    A13522006_30
151621     0.0  A13522006    A13522006_50
        prices apart_code apart_code_area
329458     0.0  A14320002   A14320002_150
329461     0.0  A14320002   A14320002_130
329462     0.0  A14320002   A14320002_160
        prices apart_code apart_code_area
723994     0.0  A13778209    A13778209_60
723996     0.0  A13778209    A13778209_50
        prices apart_code apart_code_area
655604     0.0  A11087601    A11087601_80
655605     0.0  A11087601   A11087601_110
        prices apart_code apart_code_area
409165     0.0  A15609502    A15609502_60
409167     0.0  A15609502    A15609502_80
409169     0.0  A15609502   A15609502_110
        prices apart_code apart_code_area
628605     0.0  A13583301    A13583301

9월 추가

In [ ]:
df_predction_9 = make_future_data(9)
df_a = pd.concat([df_a,df_predction_9], ignore_index=False)
df_a = fill_nan(['YYYYMM', 'YYYY', 'MM','GPS_X','GPS_Y'],'crime_rate',df_a)
filling_list = ['ASPI','HAI', 'PIR', 'LIR', 'GDP', 'GDI', 'GNI', 'CPI','avg_interest', 'kospi', 'M2','house_CCSI']
for i in filling_list:
    df_a = fill_nan(['YYYYMM', 'YYYY', 'MM'],i,df_a)
df_a = fill_nan(['YYYYMM', 'YYYY', 'MM'],'unsold',df_a)

0         201201
1         201201
2         201201
3         201201
4         201202
           ...  
725040    202208
725041    202208
725044    202208
725045    202208
725047    202208
Name: YYYYMM, Length: 724270, dtype: int64

In [ ]:
for i in test_codes:
    gizun_code = i
    gizun = df_range[df_range['apart_code']==gizun_code]
    #변수 변경시 거리 조정
    near_df = finding_near_apt(gizun,2)
    near_df = pd.merge(near_df,many_df,left_on='apart_code',right_on='k-아파트코드',how='left')
    near_df = near_df[['apart_code','k-전체세대수']]
    gizun_count = near_df[near_df['apart_code']==gizun_code].iloc[0][1]
    indexNames = near_df[near_df['k-전체세대수'] < gizun_count].index
    near_df.drop(indexNames , inplace=True)
    df = df_a
    df = pd.merge(near_df,df,on='apart_code')
    df.pop('k-전체세대수')
    df = df.drop_duplicates(subset=['apart_code_area','YYYYMM'])
    df = df.sort_values(by=["apart_code_area", "YYYYMM"], ascending=[True, True])
    df = df[['apart_code_area', 'YYYYMM','sale_type', 'area_m3', 'floor', 'const_year', 
    'parking','crime_rate', 'building_year', 'market_dist','busstop_dist', 'subway_dist', 
    'kinder_dist', 'hospital_dist', 'ASPI','YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 
    'GNI', 'CPI','avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold','near_element', 
    'near_middle', 'near_high', 'generation','const_rank','prices']]
    codes = df['apart_code_area'].unique()
    groups = df.groupby(df.apart_code_area)
    for j in codes:
        globals()['df_'+j] = groups.get_group(j)
    gizun_code_areas = df[df['apart_code_area'].str.contains(gizun_code)]['apart_code_area'].unique()
    for k in gizun_code_areas:
        gizun_code_area = k
        gizun_df = globals()['df_'+gizun_code_area]
        df_all = pd.DataFrame()
        deleted_code = 'df_'+gizun_code_area
        codes = np.delete(codes, np.where(codes == deleted_code))
        df_all = get_from_codes(codes)
        train_set = df_all[df_all['var1(t)']<202209]
        df_test = gizun_df
        values = get_float_values(df_test)
        reframed = series_to_supervised(values, 3, 1)
        reframed = delete_unuse_var(reframed)
        #날짜를 사용해 df와 df_prediction_8 구분
        test_set = reframed[reframed['var1(t)']>=202209]
        df_set = pd.concat([train_set,test_set])
        train_days =  len(train_set)
        values= df_set.values
        scaled = scaler.fit_transform(values)
        values = scaled
        train_days =  len(train_set)
        #테스트 데이터 얼마나 사용할 건지
        train = values[:train_days]
        test = values[train_days:]
        # split into input and outputs
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        history = model.fit(train_X, train_y, epochs=50, batch_size=72, 
                validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # make a prediction
        try:
            yhat = model.predict(test_X)
            test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
            inv_yhat = concatenate((test_X[:, 0:],yhat), axis=1)
            # invert scaling for actual
            inv_yhat = scaler.inverse_transform(inv_yhat)
            inv_yhat = inv_yhat[:,-1]
            #만약 해당 값을 데이터 프레임에 넣고 싶다면
            df_a['prices'][(df_a['apart_code_area']==k)&df_a['YYYYMM']==202209] = float(inv_yhat[0])
        except:
            #만약 해당 아파트의 2020년 이후 거래내역이 존재하지 않을 경우 오류 발생
            print('오류 발생') 

In [ ]:
df_predction_8['prices'].sum()

2715009.34375